<a href="https://colab.research.google.com/github/haedal-uni/analysis/blob/main/work/2025-04/Robo-Advisor/%EB%A1%9C%EB%B3%B4%EC%96%B4%EB%93%9C%EB%B0%94%EC%9D%B4%EC%A0%80%20with%20Python%20%EC%8B%A4%ED%96%89%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

해당 [링크](https://github.com/RAAILab/PyRA)를 보고 작성

In [50]:
import warnings
import matplotlib
import sys
import numpy as np
import os
import pandas as pd
warnings.filterwarnings(action='ignore')
if 'google.colab' in sys.modules:
    !echo 'debconf debconf/frontend select Noninteractive' | debconf-set-selections
    # 나눔 폰트를 설치
    !sudo apt-get -qq -y install fonts-nanum
    import matplotlib.font_manager as fm
    font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
    for fpath in font_files:
        fm.fontManager.addfont(fpath)
matplotlib.rcParams['font.family'] = 'NanumGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

Selecting previously unselected package fonts-nanum.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [51]:
!pip install pykrx
# !pip install pypfopt
!pip install pyportfolioopt

In [52]:
from pykrx import stock
import time
ticker_list = ['005930', '000020', '035720']
for ticker in ticker_list :
  df = stock.get_market_ohlcv('20181210','20181212', ticker)
  print(df.head())
  time.sleep(1)

               시가     고가     저가     종가       거래량       등락률
날짜                                                        
2018-12-10  40450  40650  40000  40200  14892263 -1.831502
2018-12-11  40600  40700  40200  40250  10638766  0.124378
2018-12-12  40250  40700  40150  40450  12024279  0.496894
              시가    고가    저가    종가    거래량       등락률
날짜                                                 
2018-12-10  9590  9710  9520  9660  58399 -0.309598
2018-12-11  9660  9760  9320  9320  82378 -3.519669
2018-12-12  9330  9580  9330  9570  23962  2.682403
               시가     고가     저가     종가     거래량       등락률
날짜                                                      
2018-12-10  22679  22779  22077  22279  331087 -3.058916
2018-12-11  22076  22177  21375  21476  461496 -3.604291
2018-12-12  21475  22277  21174  22078  393740  2.803129


In [53]:
## data_loader.py
import pandas as pd
import time
import numpy as np
# import exchange_calendars

from tqdm import tqdm
from pykrx import stock
from typing import Optional, List


class PykrxDataLoader:
    def __init__(self, fromdate: str, todate: str, market: str = "KOSPI"):
        self.fromdate = fromdate
        self.todate = todate
        self.market = market

    # 주가 데이터 불러오기
    def load_stock_data(self, ticker_list: List, freq: str, delay: float = 1):
        ticker_data_list = []
        for ticker in ticker_list:
            ticker_data = stock.get_market_ohlcv(fromdate=self.fromdate,
                                                 todate=self.todate,
                                                 ticker=ticker,
                                                 freq='d',
                                                 adjusted=True)
            ticker_data = ticker_data.rename(
                columns={'시가': 'open', '고가': 'high', '저가': 'low',
                         '종가': 'close', '거래량': 'volume',
                         '거래대금': 'trading_value', '등락률': 'change_pct'}
            )
            ticker_data = ticker_data.assign(ticker=ticker)
            ticker_data.index.name = 'date'
            ticker_data_list.append(ticker_data)
            time.sleep(delay)

        data = pd.concat(ticker_data_list)


        # 잠시 거래를 중단한 주식의 시가, 고가, 저가 보충
        data.loc[data.open == 0,
                 ['open', 'high', 'low']] = data.loc[data.open == 0, 'close']


        # 샘플링을 통해 일 데이터를 다른 주기 데이터로 변환
        if freq != 'd':
            rule = {
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum',
                # 'trading_value': 'sum',
            }
            data = data.groupby('ticker').resample(freq).apply(rule).reset_index(level=0)
        data.__setattr__('frequency', freq)
        return data



      # 인덱스 데이터 불러오기
    def load_index_data(self, ticker_list: List, freq: str, delay: float = 1):
        ticker_data_list = []
        for ticker in ticker_list:
            ticker_data = stock.get_index_ohlcv(fromdate=self.fromdate,
                                                todate=self.todate,
                                                ticker=ticker,
                                                freq=freq)
            ticker_data = ticker_data.rename(
                columns={'날짜': 'date', '시가': 'open', '고가': 'high', '저가': 'low',
                         '종가': 'close', '거래량': 'volume', '거래대금': 'trading_value',
                         '등락률': 'change_pct', '상장시가총액': 'market_cap'}
            )
            ticker_data = ticker_data.assign(ticker=ticker)
            ticker_data.index.name = 'date'
            ticker_data_list.append(ticker_data)
            time.sleep(delay)
        data = pd.concat(ticker_data_list)

        return data

In [54]:
fromdate = '2020-01-01'
todate = '2020-12-31'
ticker_list = ['005930', '000020', '035720']

data_loader = PykrxDataLoader(fromdate = fromdate, todate = todate, market='KOSPI')
ohlcv_data = data_loader.load_stock_data(ticker_list = ticker_list, freq='m', delay=1)
ohlcv_data.head(15)

,ticker,open,high,low,close,volume
date,,,,,,
2020-01-31,000020,8340,8960,7830,7910,3671841
2020-02-29,000020,7790,7890,6590,6830,2983877
2020-03-31,000020,6740,7330,4800,6550,4388718
2020-04-30,000020,6610,12450,6420,11250,32556497
2020-05-31,000020,10650,12450,9710,11550,22757189
2020-06-30,000020,11750,18100,10950,16250,80711715
2020-07-31,000020,16100,27000,14400,24400,55867988
2020-08-31,000020,24700,34450,20700,23450,59577001
2020-09-30,000020,23500,29600,20600,21850,28309516


In [55]:
def calculate_return(ohlcv_data: pd.DataFrame):
    close_data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date'])
    close_data = close_data.unstack(level=0)
    close_data = close_data['close']
    return_data = close_data.pct_change(1) * 100
    return return_data

In [56]:
from typing import Dict, Optional
from pypfopt import EfficientFrontier
def get_mean_variance_weights(return_data: pd.DataFrame,
                              risk_aversion: int) -> Optional[Dict]:
    # 수익률 계산
    expected_return = return_data.mean(skipna=False).to_list()
    # 공분산 행렬 계산
    cov = return_data.cov(min_periods=len(return_data))

    if cov.isnull().values.any() or cov.empty:
        return None

    # 평균-분산 최적화
    ef = EfficientFrontier(
        expected_returns=expected_return,
        cov_matrix=cov,
        solver='OSQP'
    )
    ef.max_quadratic_utility(risk_aversion=risk_aversion)

    # 0에 가까운 편입비중 처리
    weights = dict(ef.clean_weights(rounding=None))
    return weights

In [57]:
from typing import Optional
from enum import Enum
import datetime
import uuid

# 주문 전략
class OrderType(Enum):
    MARKET = 1 # 시장가 주문
    LIMIT = 2 # 지정가 주문
    STOPMARKET = 3 # 정지 시장가 주문
    STOPLIMIT = 4 # 정지 지정가 주문

# 거래 상태
class OrderStatus(Enum):
    OPEN = 1 # 미체결(혹은 부분 체결)
    FILLED = 2 # 완료
    CANCELLED = 3 # 취소

# 거래 방향
class OrderDirection(Enum):
    BUY = 1 # 매도
    SELL = -1 # 매수


class Order(object):
    def __init__(self, dt: datetime.date, ticker: str, amount: int,
                 type: Optional[OrderType] = OrderType.MARKET,
                 limit: Optional[float] = None, stop: Optional[float] = None,
                 id: Optional[str] = None) -> None:
        self.id = id if id is not None else uuid.uuid4().hex
        self.dt = dt
        self.ticker = ticker # 거래 종목의 종목 코드
        self.amount = abs(amount) # 주문 수량
        self.direction = OrderDirection.BUY if amount > 0 else OrderDirection.SELL # 거래 방향
        self.type = type # 주문 유형
        self.limit = limit # 지정가 여부
        self.stop = stop # 정지 주문 여부

        self.status: OrderStatus = OrderStatus.OPEN # 주문 상태
        self.open_amount: int = self.amount # 미체결 수량

In [58]:
with open("config.py", "w") as f:
    f.write("""
commission_rate = 0.0002
slippage_rate = 0.001
volume_limit_rate = 0.1
""")
import config

In [59]:
class Transaction(object):
    def __init__(self, id: str, dt: datetime.date, ticker: str, amount: int,
                 price: float, direction: OrderDirection,
                 commission_rate: float = config.commission_rate) -> None:
        self.id = id # 주문 id
        self.dt = dt # 날짜
        self.ticker = ticker # 거래 종목의 종목 코드
        self.amount = amount # 거래 수량(절댓값)
        self.price = price # 거래 가격
        self.direction = direction # 거래 방향
        self.commission_rate = commission_rate # 거래 수수료 비율

        self.commission = (self.amount * self.price) * self.commission_rate # 거래 수수료(자동 계산)
        self.settlement_value = -self.direction.value * (self.amount * self.price
                                                         ) - self.commission # 정산 금액(자동 계산)

In [60]:
from typing import List, Dict, Optional, Tuple
import datetime
import pandas as pd


class Broker(object):
    def __init__(self, slippage_rate: float = config.slippage_rate,
                 volume_limit_rate: float = config.volume_limit_rate):
        self.slippage_rate = slippage_rate # 슬리피지 비율
        self.volume_limit_rate = volume_limit_rate # 거래량 제한 비율

    # 슬리피지 계산
    def calculate_slippage(self, data: Dict, order: Order) -> Tuple[float, int]: # 가격 데이터(data)와 주문(order)을 입력 받아서 거래 체결 가격과 체결량을 반환
        # 슬리피지를 포함한 거래 가격 계산
        price = data['open']
        simulated_impact = price * self.slippage_rate

        if order.direction == OrderDirection.BUY:
            impacted_price = price + simulated_impact
        else:
            impacted_price = price - simulated_impact

        # 거래가 가능한 수량 계산
        volume = data['volume']
        max_volume = volume * self.volume_limit_rate
        shares_to_fill = min(order.open_amount, max_volume)

        return impacted_price, shares_to_fill


    # 주문 처리 및 거래 반환
    def process_order(self, dt: datetime.date, data: pd.DataFrame,
                      orders: Optional[List[Order]]) -> List[Transaction]:
        if orders is None:
            return []

        # 가격 데이터를 딕셔너리로 변환
        data = data.set_index('ticker').to_dict(orient='index')

        transactions = []
        for order in orders:
            if order.status == OrderStatus.OPEN:
                assert order.ticker in data.keys()
                # 슬리피지 계산
                price, amount = self.calculate_slippage(
                    data=data[order.ticker],
                    order=order
                )
                if amount != 0:
                    # 거래 객체 생성
                    transaction = Transaction(
                        id=order.id,
                        dt=dt,
                        ticker=order.ticker,
                        amount=amount,
                        price=price,
                        direction=order.direction,
                    )
                    transactions.append(transaction)
                    # 거래 객체의 상태와 미체결 수량 업데이트
                    if order.open_amount == transaction.amount:
                        order.status = OrderStatus.FILLED
                    order.open_amount -= transaction.amount

        return transactions

In [61]:
class AssetPosition(object):
    def __init__(self, ticker: str, position: int, latest_price: float, cost: float):
        self.ticker = ticker # 종목 코드
        self.position = position # 보유 수량(부호 있음)
        self.latest_price = latest_price # 최신 가격
        self.cost = cost # 평균 초기 가격

        self.total_settlement_value = (-1.0) * self.position * self.cost # 모든 거래의 총 정산 금액

    def update(self, transaction: Transaction):
        self.total_settlement_value += transaction.settlement_value
        self.position += transaction.direction.value * transaction.amount
        self.cost = (-1.0) * self.total_settlement_value / self.position \
            if self.position != 0 else 0.0

In [62]:
from typing import List, Dict
import datetime
import pandas as pd

class Account(object):
    def __init__(self, initial_cash: float) -> None:
        self.initial_cash = initial_cash # 초기 자금
        self.current_cash = initial_cash # 현재 자금

        self.dt = None # 날짜

        self.portfolio: Dict[str, AssetPosition] = {} # 투자 포트폴리오
        self.orders: List[Order] = [] # 현재 주문 목록

        self.transaction_history: List[Dict] = [] # 거래 히스토리
        self.portfolio_history: List[Dict] = [] # 투자 포트폴리오 히스토리
        self.account_history: List[Dict] = [] # 장부 금액 히스토리

        self.order_history: List[Dict] = [] # 주문 히스토리
        self.weight_history: List[Dict] = [] # 총자산

    @property
    def total_asset(self) -> float:
        # 현재 총 자산 계산
        market_value = 0
        for asset_position in self.portfolio.values():
            market_value += asset_position.latest_price * asset_position.position
        return market_value + self.current_cash

    def update_position(self, transactions: List[Transaction]):
        for tran in transactions:
            asset_exists = tran.ticker in self.portfolio.keys()
            if asset_exists:
                # 기존에 보유 중인 자산 포지션 업데이트
                self.portfolio[tran.ticker].update(transaction=tran)
            else:
                # 처음 보유하는 자산 추가
                new_position = AssetPosition(
                    ticker=tran.ticker, position=tran.direction.value*tran.amount,
                    latest_price=tran.price, cost=abs(tran.settlement_value)/tran.amount
                )
                self.portfolio[tran.ticker] = new_position
            # 현재 현금 업데이트
            self.current_cash += tran.settlement_value
            # 거래 히스토리 업데이트
            self.transaction_history.append(vars(tran))

    def update_portfolio(self, dt: datetime.date, data: pd.DataFrame):
        # 가격 데이터를 딕셔너리로 변환
        data = data.set_index('ticker').to_dict(orient='index')

        # 자산의 최신 가격 업데이트
        for asset_position in self.portfolio.values():
            assert asset_position.ticker in data.keys()
            asset_position.latest_price = data[asset_position.ticker]['close']

        # 투자 포트폴리오 히스토리 업데이트 (현금과 자산)
        self.portfolio_history.append(
            {'date': dt, 'ticker': 'cash', 'latest_price': self.current_cash})
        self.portfolio_history.extend(
            [{'date': dt} | vars(asset_position)
             for asset_position in self.portfolio.values()])
        # 장부 금액 히스토리 업데이트
        self.account_history.append(
            {'date': dt, 'current_cash': self.current_cash,
             'total_asset': self.total_asset})

    def update_order(self):
        # 완료 상태의 주문
        filled_orders = [order for order in self.orders
                         if order.status == OrderStatus.FILLED]
        # 주문 히스토리 업데이트
        self.order_history.extend([vars(order) for order in filled_orders])

        # 미완료 상태의 주문은 현재 주문으로 유지
        open_orders = [order for order in self.orders
                       if order.status == OrderStatus.OPEN]
        self.orders[:] = open_orders

    # def update_weight(self, dt: datetime.date, weight: dict):
    #     new_weight = weight.copy()
    #     new_weight['date'] = dt
    #     self.weight_history.append(new_weight)


In [63]:
##### 목표 수량에 따라 주문
def order_target_amount(account: Account, dt: datetime.date,
                        ticker: str, target_amount: int) -> Optional[Order]:
    # 투자 포트폴리오의 각 자산 및 보유 수량
    positions = {asset_position.ticker: asset_position.position
                 for asset_position in account.portfolio.values()}
    # 자산의 보유 수량
    position = positions.get(ticker, 0)
    # 거래 수량 계산
    amount = target_amount - position
    if amount != 0:
        # 주문 객체 생성
        return Order(dt=dt, ticker=ticker, amount=amount)
    else:
        return None

In [64]:
##### 목표 수량 계산
def calculate_target_amount(account: Account, ticker: str,
                            target_percent: float, data: pd.DataFrame) -> int:
    assert ticker in data['ticker'].to_list()
    # 총 자산
    total_asset = account.total_asset
    # 자산의 현재 가격
    price = data.loc[data['ticker'] == ticker, 'close'].squeeze()
    # 목표 보유 수량 계산
    target_amount = int(np.fix(total_asset * target_percent / price))
    return target_amount

In [65]:
##### 목표 편입 비중에 따라 구분
def order_target_percent(account: Account, dt: datetime.date, ticker: str,
                         target_percent: float, data: pd.DataFrame) -> Optional[Order]:
    # 목표 보유 수량 계산
    target_amount = calculate_target_amount(account=account, ticker=ticker,
                                            target_percent=target_percent, data=data)
    # 목표 수량에 따라 주문
    return order_target_amount(account=account, dt=dt, ticker=ticker,
                               target_amount=target_amount)

In [66]:
##### 리밸런싱
def rebalance(dt: datetime.date, data: pd.DataFrame, account: Account, weights: Dict):
    for asset_position in account.portfolio.values():
        if asset_position.ticker not in weights.keys():
            # 포트폴리오에 더 이상 포함되지 않는 기존 자산 매도
            order = order_target_percent(account=account, dt=dt,
                                         ticker=asset_position.ticker,
                                         target_percent=.0, data=data)
            # 주문 리스트에 생성된 주문 추가
            if order is not None:
                account.orders.append(order)

    for ticker, target_percent in weights.items():
        # 자산을 목표 편입비중으로 조정
        order = order_target_percent(account=account, dt=dt, ticker=ticker,
                                     target_percent=target_percent, data=data)
        if order is not None:
            # 주문 리스트에 생성된 주문 추가
            account.orders.append(order)

In [67]:
def simulate_mean_variance(ohlcv_data: pd.DataFrame, look_back: int):
    account = Account(initial_cash=100000000)
    broker = Broker()

    # 수익률 계산
    return_data = calculate_return(ohlcv_data=ohlcv_data)

    # for date, ohlcv in ohlcv_data.groupby(['date']): ############ 기존
    #     print(date.date())

    for date, ohlcv in ohlcv_data.groupby(ohlcv_data.index): ############# 변경
        print(date.date())

        # 주문 처리 및 거래 생성
        transactions = broker.process_order(dt=date, data=ohlcv, orders=account.orders)
        # 계좌 내 자산 포지션, 투자 포트폴리오, 주문 업데이트
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)
        account.update_order()

        # 현재 날짜의 수익률 데이터
        return_data_slice = return_data.loc[:date].iloc[-look_back:]
        # 자산편입비중 계산 및 뒤처리
        weights = get_mean_variance_weights(return_data=return_data_slice,
                                            risk_aversion=3)
        rounded_weights = (None if weights is None else
                           {k: round(v, 3) for k, v in weights.items()})
        print(f'Portfolio: {rounded_weights}')
        if weights is None:
            continue

        # 투자 포트폴리오 조정
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    return account

In [68]:
def get_lookback_fromdate(fromdate: str, lookback: int, freq: str) -> str:
    # freq에 따라 룩백 기간 포함된 예상 시작 날짜를 설정
    if freq == 'd':
        estimated_start_date = pd.to_datetime(fromdate) - pd.DateOffset(days=lookback*2)
    elif freq == 'm':
        estimated_start_date = pd.to_datetime(fromdate) - pd.DateOffset(months=lookback)
    elif freq == 'y':
        estimated_start_date = pd.to_datetime(fromdate) - pd.DateOffset(years=lookback)
    else:
        raise ValueError
    # 설정 기간(estimated_start_date ~ fromdate)의 KOSPI 데이터를 다운로드
    kospi = stock.get_index_ohlcv(fromdate=str(estimated_start_date.date()),
                                  todate=fromdate, ticker='1001', freq=freq)
    # 룩백 기간을 포함하는 정확한 시작 날짜를 반환
    return str(kospi.index[-lookback].date())

In [69]:
##### 시뮬레이션 스크립트(지정된 구간 동안의 평균-분산 전략 시뮬레이션 결과를 얻을 수 있다.)
fromdate = '2020-07-10'
todate = '2023-09-27'
ticker_list =  ['005930', '000660', '207940', '051910',
                '006400', '005380', '000270', '005490', '035420']

total_look_back = 1 + 24
adj_fromdate = get_lookback_fromdate(fromdate=fromdate,
                                     lookback=total_look_back, freq='m')

data_loader = PykrxDataLoader(fromdate=adj_fromdate,
                              todate=todate, market='KOSPI')
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list,
                                         freq='m', delay=1)
ohlcv_data.head(15)

,ticker,open,high,low,close,volume
date,,,,,,
2018-07-31,000270,31800,31950,31500,31700,636196
2018-08-31,000270,31650,32600,30950,32050,13377137
2018-09-30,000270,31900,35600,31350,35100,19573161
2018-10-31,000270,35150,35200,26200,28400,27693336
2018-11-30,000270,28300,30750,27100,30500,19094252
2018-12-31,000270,30900,34200,29600,33700,16249408
2019-01-31,000270,33450,36650,31800,36350,21942579
2019-02-28,000270,36350,37350,34750,36550,15087190
2019-03-31,000270,36500,36500,33350,35350,18778392


In [70]:
simulation_account = simulate_mean_variance(ohlcv_data=ohlcv_data, look_back=24)

2018-07-31
Portfolio: None
2018-08-31
Portfolio: None
2018-09-30
Portfolio: None
2018-10-31
Portfolio: None
2018-11-30
Portfolio: None
2018-12-31
Portfolio: None
2019-01-31
Portfolio: None
2019-02-28
Portfolio: None
2019-03-31
Portfolio: None
2019-04-30
Portfolio: None
2019-05-31
Portfolio: None
2019-06-30
Portfolio: None
2019-07-31
Portfolio: None
2019-08-31
Portfolio: None
2019-09-30
Portfolio: None
2019-10-31
Portfolio: None
2019-11-30
Portfolio: None
2019-12-31
Portfolio: None
2020-01-31
Portfolio: None
2020-02-29
Portfolio: None
2020-03-31
Portfolio: None
2020-04-30
Portfolio: None
2020-05-31
Portfolio: None
2020-06-30
Portfolio: None
2020-07-31
Portfolio: {'000270': 0.0, '000660': 0.333, '005380': 0.033, '005490': 0.107, '005930': 0.127, '006400': 0.215, '035420': 0.031, '051910': 0.103, '207940': 0.051}
2020-08-31
Portfolio: {'000270': 0.0, '000660': 0.323, '005380': 0.009, '005490': 0.122, '005930': 0.151, '006400': 0.226, '035420': 0.032, '051910': 0.091, '207940': 0.046}
2020

In [71]:
# Data utility
##### 종목 코드에 해당하는 종목 이름을 가져오는 함수
def ticker_to_name(ticker: str) -> str:
    if ticker == 'cash':
        return '현금'
    else:
        return stock.get_market_ticker_name(ticker=ticker)

In [72]:
df_account = pd.DataFrame(simulation_account.account_history).set_index('date')
df_portfolio = pd.DataFrame(simulation_account.portfolio_history).set_index('date')
df_portfolio = df_portfolio.assign(
    ticker=df_portfolio['ticker'].apply(lambda x: f'{x}({ticker_to_name(x)})')
)

In [73]:
analysis_fromdate = df_account.index[total_look_back-1]
df_portfolio = df_portfolio.loc[analysis_fromdate:]

In [74]:
returns = df_account['total_asset'].pct_change().loc[analysis_fromdate:]
returns.name = 'return'

In [75]:
kospi = data_loader.load_index_data(ticker_list=['1001'], freq='m', delay=1)
kospi_returns = kospi['close'].pct_change().loc[analysis_fromdate:]
kospi_returns.iloc[0] = 0.0
kospi_returns.name = 'kospi_return'
kospi_returns.index.name = 'date'

In [76]:
pd.concat([returns, kospi_returns], axis=1)

,return,kospi_return
date,,
2020-07-31,0.000000,0.000000
2020-08-31,0.033602,0.034143
2020-09-30,0.030677,0.000739
2020-10-31,-0.017236,-0.026092
2020-11-30,0.132208,0.142995
2020-12-31,0.123685,0.108874
2021-01-31,0.081543,0.035755
2021-02-28,0.088482,0.012345
2021-03-31,-0.003068,0.016087


In [77]:
##### 주기별 연 환산 인자
annualization_factor = {
    'd' : 252,
    'm' : 12,
    'y' : 1
}

In [78]:
def cagr(returns: pd.Series, freq: str = 'd') -> float:
    if len(returns) < 1:
        return np.nan
    ann_factor = annualization_factor[freq]
    num_years = len(returns) / ann_factor
    cum_return = (returns + 1).prod()
    return cum_return ** (1 / num_years) - 1


In [79]:
cagr(returns=returns, freq='m')

np.float64(0.0759645037107024)

In [80]:
cagr(returns=kospi_returns, freq='m')

np.float64(0.028576073770819388)

In [81]:
### mdd 함수는 수익률 데이터(returns)를 입력 파라미터로 받아 MDD를 반환한다.
def mdd(returns: pd.Series) -> float:
    if len(returns) < 1:
        return np.nan
    cum_returns = (returns + 1).cumprod()
    max_return = np.fmax.accumulate(cum_returns, axis=0)
    mdd = ((cum_returns - max_return) / max_return).min()
    return mdd

In [82]:
mdd(returns=returns)

-0.28639601118989266

In [83]:
mdd(returns=kospi_returns)

-0.34616341288811797

In [84]:
def sharpe_ratio(returns: pd.Series, risk_free: float = 0,
                 freq: str = 'd') -> float:
    if len(returns) < 2:
        return np.nan
    adjusted_returns = returns - risk_free
    ann_factor = annualization_factor[freq]
    sharpe_ratio = (adjusted_returns.mean() / adjusted_returns.std()
                    * np.sqrt(ann_factor))
    return sharpe_ratio

In [85]:
sharpe_ratio(returns=returns, freq='m')

np.float64(0.4607683501443832)

In [86]:
sharpe_ratio(returns=kospi_returns, freq='m')

np.float64(0.24007555382922374)

In [87]:
def sortino_ratio(returns: pd.Series, risk_free: float = 0,
                 freq: str = 'd') -> float:
    if len(returns) < 2:
        return np.nan
    adjusted_returns = returns - risk_free
    negative_returns = adjusted_returns[adjusted_returns < 0]
    downside_risk = np.sqrt(np.mean(negative_returns.pow(2)))
    ann_factor = annualization_factor[freq]
    if downside_risk == 0:
        sortino_ratio = np.NAN
    else:
        sortino_ratio = (adjusted_returns.mean() / downside_risk
                        * np.sqrt(ann_factor))
    return sortino_ratio

In [88]:
sortino_ratio(returns=returns, freq='m')

np.float64(0.4217295847537774)

In [89]:
sortino_ratio(returns=kospi_returns, freq='m')

np.float64(0.2195446996598145)

In [90]:
# coding: utf-8
import pandas as pd
from plotly import graph_objects as go, express as px

def plot_cumulative_return(returns: pd.Series, benchmark_returns: pd.Series,
                           strategy_name: str = 'My Strategy',
                           benchmark_name: str = 'KOSPI') -> None:
    # 포트폴리오의 누적 수익률 계산
    cum_returns = (returns + 1).cumprod() - 1

    # KOSPI의 누적 수익률 계산
    benchmark_cum_returns = (benchmark_returns + 1).cumprod() - 1

    # 그래프 객체
    fig = go.Figure()

    # 포트폴리오의 누적 수익률 곡선
    fig.add_trace(go.Scatter(x=cum_returns.index, y=cum_returns,
                             name=strategy_name))

    # KOSPI의 누적 수익률 곡선
    fig.add_trace(go.Scatter(x=benchmark_cum_returns.index, y=benchmark_cum_returns,
                             name=benchmark_name, line = dict(dash='dot')))

    # 날짜 표시 형식
    fig.update_xaxes(tickformat='%Y-%m-%d')

    # 그래프 설정
    fig.update_layout(
        width=800,
        height=400,
        xaxis_title='날짜',
        yaxis_title='누적 수익률',
        legend_title_text='포트폴리오',
    )
    fig.show()

In [91]:
plot_cumulative_return(returns=returns, benchmark_returns=kospi_returns,
                       strategy_name='평균-분산',
                       benchmark_name='KOSPI')

In [92]:
def plot_single_period_return(returns: pd.Series,
                              benchmark_returns: pd.Series,
                              strategy_name: str = 'My Strategy',
                              benchmark_name: str = 'KOSPI') -> None:
    fig = go.Figure()
    fig.add_trace(go.Bar(x=returns.index, y=returns,
                         name=strategy_name))
    fig.add_trace(go.Bar(x=benchmark_returns.index, y=benchmark_returns,
                         name=benchmark_name, marker_pattern_shape='/'))
    fig.update_xaxes(tickformat='%Y-%m-%d')
    fig.update_layout(
        width=800,
        height=400,
        xaxis_title='날짜',
        yaxis_title='수익률',
        legend_title_text='포트폴리오',
    )
    fig.show()

In [93]:
plot_single_period_return(returns=returns, benchmark_returns=kospi_returns,
                          strategy_name='평균-분산',
                          benchmark_name='KOSPI')

In [94]:
def plot_relative_single_period_return(returns: pd.Series,
                                       benchmark_returns: pd.Series) -> None:
    relative_returns = returns - benchmark_returns

    fig = go.Figure()
    fig.add_trace(go.Bar(x=relative_returns.index, y=relative_returns))
    fig.update_xaxes(tickformat='%Y-%m-%d')
    fig.update_layout(
        width=800,
        height=400,
        xaxis_title='날짜',
        yaxis_title='상대 수익률',
        legend_title_text=None,
    )
    fig.show()

In [95]:
plot_relative_single_period_return(returns=returns, benchmark_returns=kospi_returns)

In [96]:
def plot_cumulative_asset_profit(df_portfolio: pd.DataFrame) -> None:
    df_portfolio = df_portfolio.assign(
        profit=((df_portfolio['latest_price'] - df_portfolio['cost'])
                * df_portfolio['position']).fillna(0)
    )
    df_asset_profit = df_portfolio[['ticker', 'profit']].set_index(
        'ticker', append=True).unstack(level=-1, fill_value=0)['profit']
    df_asset_position = df_portfolio[['ticker', 'position']].set_index(
        'ticker', append=True).unstack(level=-1, fill_value=0)['position']
    df_asset_profit_change = df_asset_profit.diff()
    df_asset_profit_change[df_asset_position==0] = 0
    df_asset_cumulative_profit = df_asset_profit_change.cumsum()

    from plotly.validators.scatter.marker import SymbolValidator
    raw_symbols = SymbolValidator().values[2::12]

    fig = go.Figure()
    for idx, col in enumerate(df_asset_cumulative_profit.columns):
        if 'cash' in col:
            continue
        fig.add_trace(go.Scatter(x=df_asset_cumulative_profit.index,
                                 y=df_asset_cumulative_profit[col],
                                 name=col, mode='lines+markers',
                                 marker={'symbol': raw_symbols[idx]}))
    fig.update_xaxes(tickformat='%Y-%m-%d')
    fig.update_layout(
        width=800,
        height=400,
        xaxis_title='날짜',
        yaxis_title='누적 수익률',
        legend_title_text='종목코드',
    )
    fig.show()

In [97]:
plot_cumulative_asset_profit(df_portfolio=df_portfolio)

In [98]:
#### 자산 편입 비중 영역 차트 그림 및 채우기 도형 적용

def plot_asset_weight(df_portfolio: pd.DataFrame) -> None:
    # 현금의 보유수량을 1로 설정
    df_portfolio = df_portfolio.assign(
        position=df_portfolio['position'].fillna(1)
    )
    # 자산의 시가총액 계산
    df_portfolio = df_portfolio.assign(
        value=df_portfolio['latest_price'] * df_portfolio['position']
    )
    # 자산의 편입비중 계산
    df_portfolio = df_portfolio.assign(
        weight=df_portfolio['value'].groupby('date').transform(lambda x: x / x.sum())
    )

    fig = px.area(data_frame=df_portfolio, y='weight',
                  color='ticker', pattern_shape='ticker')
    fig.update_xaxes(tickformat='%Y-%m-%d')
    fig.update_layout(
        width=800,
        height=400,
        xaxis_title='날짜',
        yaxis_title='자산편입비중',
        legend_title_text='종목코드',
    )
    fig.show()

In [99]:
#### 각 자산 편입 비중의 변화 추이를 볼 수 있다.
plot_asset_weight(df_portfolio=df_portfolio)